In [1]:
from datetime import datetime
def time_now():
    '''Get Current Time'''
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S.%f")
    print("Current Time =", current_time)
    return now


############
print('Starting...')
start = time_now()
############


Starting...
Current Time = 21:47:57.001260


In [2]:
import regex as re

In [3]:
import swifter

In [4]:
import pandas as pd

In [5]:
import regex as re

In [6]:
from itertools import chain

In [7]:
from collections import Counter

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

from spacy.matcher import Matcher

In [9]:
import os

In [10]:
ignore_list = ['#', '’', "'"  '!',  ',',  '(',  '.',  '‡',  '9',  '*',  '2',  ']',  '†',  '1',  '"',  '/',  '5',  '4',  '0',  ';',  '¾',  '‘',  '¼',  '{',  ')',  '…',  '½',  '6',  '×',  '[',  '+',  ':',  '$',  '3',  '?',  '7',  '©',  '’',  '£',  '8']

pat_verb_noun_phrase =[
    {'POS': {'IN': 'VERB ADV'.split()}, 'OP':'+'}, 
    {'POS': {'IN' : 'DET ADP PART PRON'.split()}, 'OP':'*'},
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()}, 'OP':'+'}
]


pat_noun_verb_phrase = [
            {'POS': {'IN': ['NOUN', 'ADJ']}, 'OP': '+'},
            {'POS': {'IN': ['AUX', 'PART', 'ADP']}, 'OP': '*'},
            {'POS': {'IN': ['VERB', 'ADV']}, 'OP': '+'}
            ]
            
            

pat_n_p_v = [
            {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
            {'POS': {'IN': ['PUNCT', 'CCONJ']}, 'LEMMA': {'NOT_IN': ignore_list}, 'OP': '+'},
            {'POS': {'IN': ['NOUN', 'ADJ', 'VERB', 'PROPN']}, 'OP':'+'}
             ]            
             
pat_noun = [
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'}
                        ]

pat_n_n = [
    {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
    {'POS': {'IN': ['DET', 'ADP', 'PART']}, 'OP': '+'},
    {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'}
    ]

In [11]:
list_pos_patterns = [i for i in dir() if i.startswith('pat_')]

In [12]:
try:
    threshold_pattern = int(input('''
Enter threhold per pattern per batch:  '''))
    print(f"""
Threshold is {threshold_pattern}
    """)
except:
    print('Threshold must be an integer number; 2 was chosen for you')
    threshold_pattern = 2


Enter threhold per pattern per batch:   4



Threshold is 4
    


In [13]:
def get_pos_lemma_pattern(doc, pat_name='anything', pat_collection='list_of_dictionaries'):
    '''
    Given a spacy doc object; find the span according to the pattern given.
    ''' 
    matcher =  Matcher(vocab = nlp.vocab)
    matcher.add(f'{pat_name}', pat_collection, greedy='LONGEST')
    
    
    doc_match = matcher(doc)
    
    
    list_container = []
    for match in doc_match:
        start = match[1]
        end = match[2]
        
        result_lemma = doc[start:end]
        result_lemma = [(i.lemma_ + i.whitespace_) for i in result_lemma] # new!
        result_lemma = ''.join(result_lemma).strip()

        result_token = doc[start:end]
        result_token = [(i.text + i.whitespace_) for i in result_token] # new!
        result_token = ''.join(result_token).strip()

        result_pos = doc[start:end]
        result_pos = [i.pos_ for i in result_pos] # new!
        result_pos = ' '.join(result_pos).strip()
        
        if len(result_lemma.split()) > 1:
            list_container.append((result_token, result_lemma, result_pos))

    return list_container

In [14]:
#os.mkdir('IATA_TRANSCRIPTION_PATTERNS')

In [15]:
# for root, dirs, files in os.walk('NLP/'):
#     files = sorted(files)
#     files_path = [os.path.join(root, file) for file in files if file.endswith('.pkl')]
#     files_path = [f'./{file}' for file in files_path]

In [16]:
files_path = ['EN_STEEL.pkl']

In [17]:
from pathlib import Path
channels_dir = input('''Where to save results?
Give a name for the directory!
''')
Path(channels_dir).mkdir(parents=True, exist_ok=True)
channels_dir = re.sub(r'[\/\.]+', '', channels_dir)
print(f'Results are in {channels_dir} folder')

Where to save results?
Give a name for the directory!
 NLP_RESULTS


Results are in NLP_RESULTS folder


In [18]:
for (idx, file) in enumerate(files_path):
    print(f'Processing {file} Batch {idx+1} of {len(files_path)}')

    df_temp = pd.read_pickle(file)
    
    
    print('Starting NLP')
    
    start_nlp = time_now()
#     print('Renaming TEXT_PUNCT Column!!')
#     df_temp['TEXT'] = df_temp['TEXT_PUNCT'] # Remove this line, if necessar!!!
    
    df_temp['NLP'] = df_temp['TEXT'].swifter.apply(lambda x: nlp(x))
    
    finish_nlp = time_now()
    print('Finished NLP')
    duration  = ((finish_nlp - start_nlp).seconds)/60
    print(f'Total NLP Duration: {duration} minutes')
    
    dict_verbs_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'VERB'])))).most_common())
    dict_nouns_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'NOUN'])))).most_common())
    dict_adj_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'ADJ'])))).most_common())
    dict_adv_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'ADV'])))).most_common())
    
    pd.DataFrame(data = dict_adj_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle(F'./{channels_dir}/BATCH_{str(idx+1).zfill(2)}_ADJECTIVES.pkl', protocol=4)
    pd.DataFrame(data = dict_adv_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle(F'./{channels_dir}/BATCH_{str(idx+1).zfill(2)}_ADVERBS.pkl', protocol=4)
    pd.DataFrame(data = dict_nouns_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle(F'./{channels_dir}/BATCH_{str(idx+1).zfill(2)}_NOUNS.pkl', protocol=4)
    pd.DataFrame(data = dict_verbs_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle(F'./{channels_dir}/BATCH_{str(idx+1).zfill(2)}_VERBS.pkl', protocol=4)
    
    
    for pat in list_pos_patterns:
        pattern_name = pat.upper()
        df_temp[pattern_name] = df_temp['NLP'].swifter.apply(lambda x: get_pos_lemma_pattern(x, f'{pat}', [eval(pat)]))
        
        list_pat = list(chain(*df_temp[pattern_name]))
        df_pattern  = pd.DataFrame(data=list_pat)
        df_pattern = df_pattern.rename(columns={0: 'TEXT', 1: 'LEMMA', 2: 'POS'})
        
        dict_pattern = dict(df_pattern['LEMMA'].value_counts())
        dict_pattern = {k:v for k,v in dict_pattern.items() if v >= threshold_pattern}
        
        df_pattern = df_pattern[df_pattern['LEMMA'].swifter.apply(lambda x: x in dict_pattern)].reset_index(drop=True)
        df_pattern.to_pickle(F'./{channels_dir}/BATCH_{str(idx+1).zfill(2)}_{pattern_name}.pkl', protocol=4)
    print(f'Finished {file} Batch {idx+1} of {len(files_path)}')

Processing EN_STEEL.pkl Batch 1 of 1
Starting NLP
Current Time = 21:48:28.110243


Pandas Apply:   0%|          | 0/5191 [00:00<?, ?it/s]

Current Time = 21:49:21.372240
Finished NLP
Total NLP Duration: 0.8833333333333333 minutes


Pandas Apply:   0%|          | 0/5191 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/12532 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/5191 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8070 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5191 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/12310 [00:00<?, ?it/s]

Finished EN_STEEL.pkl Batch 1 of 1


In [19]:
df_temp

,URL,TEXT,LANG,LANG_DETECT,LANG_SCORE,NLP,PAT_N_N,PAT_N_P_V,PAT_NOUN,PAT_NOUN_VERB_PHRASE,PAT_VERB_NOUN_PHRASE
0,https://aisusteel.org/en/12537/,The modified unaudited financial results of th...,"(__label__en, 0.943916380405426)",__label__en,0.943916,"(The, modified, unaudited, financial, results,...",[(unaudited financial results of the Egyptian ...,"[(Egyptian Iron and Steel showed net losses, E...","[(unaudited financial results, unaudited finan...",[],"[(modified unaudited financial results, modify..."
1,https://aisusteel.org/en/12537/,Sales slumped to EGP 1.08 billion in the year ...,"(__label__en, 0.964730978012085)",__label__en,0.964731,"(Sales, slumped, to, EGP, 1.08, billion, in, t...","[(statement to the Egyptian Exchange, statemen...",[],"[(Egyptian Exchange, Egyptian Exchange, PROPN ...","[(Sales slumped, sale slump, NOUN VERB), (year...","[(said in a statement, say in a statement, VER..."
2,https://aisusteel.org/en/12537/,"During the first nine months of FY19/20, the c...","(__label__en, 0.9291689991950989)",__label__en,0.929169,"(During, the, first, nine, months, of, FY19/20...","[(net losses of EGP, net loss of EGP, ADJ NOUN...","[(year-ago period, year-ago period, NOUN PUNCT...","[(net losses, net loss, ADJ NOUN), (ago period...","[(company reported, company report, NOUN VERB)]","[(reported net losses, report net loss, VERB A..."
3,https://aisusteel.org/en/6518/,"Nucor Steel has awarded SMS group, Inc. the co...","(__label__en, 0.9582651257514954)",__label__en,0.958265,"(Nucor, Steel, has, awarded, SMS, group, ,, In...","[(complete melt shop for the new facility, com...",[],"[(complete melt shop, complete melt shop, ADJ ...","[(new facility to be built, new facility to be...","[(supply a complete melt shop, supply a comple..."
4,https://aisusteel.org/en/6518/,The scope includes two 190 metric ton direct c...,"(__label__en, 0.8953480124473572)",__label__en,0.895348,"(The, scope, includes, two, 190, metric, ton, ...",[],[],[(metric ton direct current electric arc furna...,"[(scope includes, scope include, NOUN VERB)]",[]
...,...,...,...,...,...,...,...,...,...,...,...
5186,https://aisusteel.org/en/10150/,Arab countries’ production of crude steel,"(__label__en, 0.6892220973968506)",__label__en,0.689222,"(Arab, countries, ’, production, of, crude, st...","[(Arab countries’ production, arab country’ pr...",[],"[(Arab countries, arab country, ADJ NOUN), (cr...",[],[]
5187,https://aisusteel.org/en/9028/,According to the data provided by the Turkish ...,"(__label__en, 0.9031760096549988)",__label__en,0.903176,"(According, to, the, data, provided, by, the, ...","[(May this year Turkey, May this year Turkey, ...",[],"[(Turkish Statistical Institute, Turkish Stati...","[(data provided, datum provide, NOUN VERB), (h...",[(provided by the Turkish Statistical Institut...
5188,https://aisusteel.org/en/9028/,The revenue from these imports amounted to $41...,"(__label__en, 0.9179209470748901)",__label__en,0.917921,"(The, revenue, from, these, imports, amounted,...","[(same month of the previous year, same month ...","[(month and increasing, month and increase, NO...","[(% month, % month, NOUN NOUN), (same month, s...","[(imports amounted, import amount, NOUN VERB),...","[(compared to the same month, compare to the s..."
5189,https://aisusteel.org/en/9028/,"Meanwhile, in the first five months of this ye...","(__label__en, 0.8969374895095825)",__label__en,0.896937,"(Meanwhile, ,, in, the, first, five, months, o...","[(months of this year, month of this year, NOU...",[],"[(HRC imports, HRC import, PROPN NOUN), (metri...","[(imports amounted, import amount, NOUN VERB),...","[(compared to the same period, compare to the ..."


In [22]:
pd.read_pickle('NLP_RESULTS/BATCH_0')

,WORD,FREQ
0,steel,2060
1,year,1905
2,%,1741
3,price,1054
4,iron,879
...,...,...
2848,will,1
2849,bond,1
2850,leap,1
2851,enhancement,1
